In [1]:
import torch as tc
import pandas as pd
from src.bbox import BBox
from tqdm import tqdm
from src.curve_utils import CurveUtils
from src.curve_generator import CurveGenerator
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
genders = ['female', 'male']

In [2]:
their_semantics = [
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Upper arm girth R',
    'Neck girth',
    'Back neck point to waist',
    'Height (m)',
]
our_semantic = [
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'thigh_girth', # 5.3.20
    'upper_arm_girth', # 5.3.16
    'neck_girth', # 5.3.2
    'neck_waist',
    'stature', # 5.1.1
]

curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
    #'neck_waist': 5
}

In [3]:
subdivided_bodies = tc.load('data/subdivided_bodies.pt')
gender_curves = tc.load("data/gender_curves.zip")
gender_results = tc.load("data/best_gender_curves.zip")

In [4]:
column_curves_measures = {}

for idx, gender in enumerate(genders):

    template = subdivided_bodies['faces'][gender].to(device)

    column_curves_measures[gender] = []
    best_neck = gender_curves[gender][curve_index['neck_girth']][gender_results[gender]['best']['neck_girth']]
    best_neck = tc.FloatTensor(best_neck).to(device)

    best_waist = gender_curves[gender][curve_index['waist_girth']][gender_results[gender]['best']['waist_girth']]
    best_waist = tc.FloatTensor(best_waist).to(device)
    
    bodies_interator = tqdm(subdivided_bodies['vertices'][gender])
    for body in bodies_interator:

        best_neck_pos = CurveUtils.generate_positions(best_neck, body.to(device))
        best_waist_pos = CurveUtils.generate_positions(best_waist, body.to(device))

        neck_ht = best_neck_pos[:,1].max()
        waist_ht = best_waist_pos[:,1].max()

        neck_width = abs(best_neck_pos[:,2]).max()
        waist_width = abs(best_waist_pos[:,2]).max()

        
        width = (neck_width + (waist_width/2))*1.2
        height = neck_ht - waist_ht
        center = (0, waist_ht + (height/2), -width/2)
        column_box = BBox(length=width, width=width, height=height, center=tc.FloatTensor(center).to(device))
        column_curves = CurveGenerator.calculate_curve(body.to(device), column_box, 0, template, device)      


        all_col_curves = []
        for curve in column_curves[0]:
            positions = CurveUtils.generate_positions(tc.FloatTensor(curve).to(device), body.to(device))
            all_col_curves.append(positions[:,0].abs().mean())
        idmin = tc.FloatTensor(all_col_curves).abs().argmin()
        measure = CurveUtils.linear_distance(tc.FloatTensor(column_curves[2][idmin]))
        column_curves_measures[gender].append(measure)

100%|██████████| 72/72 [12:12<00:00, 10.18s/it]


In [5]:
subdivided_bodies = tc.load('data/subdivided_bodies.pt')
measures = pd.read_pickle(f'data/cleaned_measures.zip')
measures.index = measures['Subject']
measures = measures[measures['Measuring station'] == "MOVE4D"]

mfd_gender_measures = dict()
for gender in genders:
    mfd_gender_measures[gender] = measures[measures['Sex'] == gender]
    mfd_gender_measures[gender] = mfd_gender_measures[gender][their_semantics]
    mfd_gender_measures[gender].columns = our_semantic
    mfd_gender_measures[gender]['stature'] *= 1000

In [6]:
errors = abs(column_curves_measures['male'] - (mfd_gender_measures['male']['neck_waist']/10))
gender_results['male'].loc['neck_waist'] = 0, errors.min(), errors.max(), errors.mean(), errors.std()

errors = abs(column_curves_measures['female'] - (mfd_gender_measures['female']['neck_waist']/10))
gender_results['female'].loc['neck_waist'] = 0, errors.min(), errors.max(), errors.mean(), errors.std()

In [7]:
(gender_results['male']*10).astype(float).round(decimals=2)

,best,min,max,mean,std
bust_chest_girth,1870.0,0.51,46.89,11.82,10.46
waist_girth,3350.0,0.48,16.50,5.69,4.00
hip_girth,1320.0,0.07,12.82,4.52,3.37
thigh_girth,5430.0,0.38,19.92,6.90,5.09
upper_arm_girth,3900.0,0.03,9.04,3.13,2.05
neck_girth,52680.0,0.17,23.89,6.93,5.74
neck_waist,0.0,0.31,32.52,8.00,7.23


In [8]:
(gender_results['female']*10).astype(float).round(decimals=2)

,best,min,max,mean,std
bust_chest_girth,1470.0,0.21,41.67,17.48,12.65
waist_girth,3010.0,0.07,35.88,10.80,9.54
hip_girth,1090.0,0.65,49.38,11.92,10.88
thigh_girth,4720.0,0.57,35.87,8.27,7.42
upper_arm_girth,3460.0,0.15,13.50,5.42,3.58
neck_girth,38000.0,0.03,12.86,4.94,3.40
neck_waist,0.0,0.45,36.92,12.72,8.00


In [9]:
gender_measures = dict()
for gender in genders:
    print(f'MEASURING {gender.upper()} BODIES', end='\n')
    
    gender_measures[gender] = []
    
    for curve_name in our_semantic[:-2]:

        
        index = curve_index[curve_name]
        best_id = gender_results[gender].loc[curve_name]['best']
        best_curve = gender_curves[gender][index][best_id]

        body_measures = []
        for body in subdivided_bodies['vertices'][gender]:
            position = CurveUtils.generate_positions(tc.FloatTensor(best_curve).to(device), body.to(device))
            calculated_measures = CurveUtils.calculate_distances(position)
            body_measures.append(calculated_measures)

        gender_measures[gender].append(body_measures)
        
    gender_measures[gender] = tc.FloatTensor(gender_measures[gender])

tc.save(gender_measures, "data/calculated_measures.zip")

MEASURING FEMALE BODIES
MEASURING MALE BODIES


In [10]:
best_gender_measures = {}

for idx, gender in enumerate(genders):
    best_gender_measures[gender] = tc.row_stack([gender_measures[gender], tc.column_stack(column_curves_measures[gender])]).numpy()
    best_gender_measures[gender] = list(best_gender_measures[gender]*10)
    best_gender_measures[gender].append(tc.arange(2).repeat(36).numpy()+1)
    best_gender_measures[gender].append([gender]*72)
    best_gender_measures[gender].append(["our"]*72)
    best_gender_measures[gender].append(mfd_gender_measures[gender].index)

In [11]:
aditional_semantic = ['repetition', "gender", "measures_station", 'subject']
our_measures = pd.concat([
    pd.DataFrame(best_gender_measures['female'], index=our_semantic[:-1]+aditional_semantic).T,
    pd.DataFrame(best_gender_measures['male'], index=our_semantic[:-1]+aditional_semantic).T
])

our_measures.to_pickle("data/our_measures.zip")

In [12]:
our_measures

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth,neck_waist,repetition,gender,measures_station,subject
0,890.102844,750.670166,1049.499756,628.432495,292.741119,325.192902,418.21109,1,female,our,IEEEP2_01
1,895.18927,753.261353,1051.522095,625.345581,294.438354,326.743317,426.129272,2,female,our,IEEEP2_01
2,903.245544,682.914734,947.78717,538.580383,274.551392,314.705322,417.136688,1,female,our,IEEEP2_03
3,886.251953,664.944397,951.866211,537.98114,274.649323,316.283325,414.10672,2,female,our,IEEEP2_03
4,1098.261108,981.130737,1141.172607,673.190918,343.884399,369.490021,431.907532,1,female,our,IEEEP2_07
...,...,...,...,...,...,...,...,...,...,...,...
67,980.984009,757.277832,948.672546,577.736755,321.789185,368.718323,426.727905,2,male,our,IEEEP2_67
68,1237.505981,1257.936035,1151.078735,671.114441,363.397705,440.467407,490.968323,1,male,our,IEEEP2_70
69,1230.704834,1269.401123,1153.59729,665.347534,362.148895,440.307922,487.573822,2,male,our,IEEEP2_70
70,951.453796,878.735229,990.817505,556.553284,318.288269,385.026276,443.301636,1,male,our,IEEEP2_72
